## Load packages

In [1]:
#!python -m spacy download en_core_web_lg
# !conda install spacy download en_core_web_lg
# !python -m spacy validate
import pandas as pd
import spacy.displacy as displacy
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm
import random

## Working with NER

In [5]:
df = pd.read_csv('ubereats_cleaned.csv', header=None)
type(df)

pandas.core.frame.DataFrame

In [6]:
df.head()

,0
0,grilled chicken & ribs box
1,chicken soup with noodles
2,double ka meetha
3,skillet cinnamon roll pint
4,xuan asian bowl


In [38]:
df[0].head(50)

0                       grilled chicken & ribs box
1                        chicken soup with noodles
2                                 double ka meetha
3                       skillet cinnamon roll pint
4                                  xuan asian bowl
5                              red bull sugar free
6                                    pork teriyaki
7                   dr pepper bbq ribs family meal
8     baked chocolate cookie topped with ice cream
9                                  banana smoothie
10                                       gallon un
11                  horizon chocolate organic milk
12                                  chicken finger
13                         broccoli w garlic sauce
14                   family bundle salmon saporito
15                                    beef sliders
16                             buffalo tender wrap
17                     german chocolate cake shake
18                                 cheesy broccoli
19                           ma

In [39]:
data = list(df[0].head(50))

In [40]:
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

## Train data

In [41]:
random.seed(42)

In [42]:
sentences = [
    'I like to eat __ at this joint',
    'The __ was fire', 
    '__ are amazing, you have to try it!', 
    'Everytime I visit, I always get __',
    'My friends and I are never getting __ again',
    'This place always messes up my order of __',
    'I think that __ is the best thing on the menu'
]


In [43]:
TRAIN_DATA = []

target_sent = ''
nbr_of_sentences = len(sentences)

for ind, val in enumerate(data):
    
    for i in range(nbr_of_sentences):
        if ind % nbr_of_sentences == i:
            target_sent = sentences[i]
            ind_start = target_sent.find('__')
            ind_end = ind_start + len(val)
            target_sent = target_sent.replace('__', val)
            break
        
        
        
#     random_sentence = gen.sentence()
#     count = random_sentence.count(' ')
#     random_space = randrange(count)
    #index = find_nth(random_sentence, ' ', random_space)
    #final_sentence = random_sentence[: index]+ ' ' + val + random_sentence[index:]
    #final_sentence.replace('\n', '')
    target_sent = target_sent.lower()
    #final_sentence = re.sub(r'[^a-z ]', '', final_sentence)
    new_format = (target_sent, {'entities':[(ind_start, ind_end, 'FOOD')]})
    TRAIN_DATA.append(new_format)

 #("Who is Shaka Khan?", {"entities": [(7, 17, "PERSON")]}),
# TRAIN_DATA = [
#     ('hot pepper chicken nuggets are best at this joint', {
#         'entities': [(0, 26, 'FOOD')]
#     }),
#      ('Turkey tacos was fire', {
#         'entities': [(0, 12, 'FOOD')]
#     }),
#     ('Pork Ribs are good, you have to try it!', {
#         'entities': [(0, 9, 'FOOD')]
#     })
# ]


#   'entities': [(7, 13, 'FOOD'), (18, 24, 'LOC')] - more than 1 entity in a sentence

In [44]:
TRAIN_DATA

[('i like to eat grilled chicken & ribs box at this joint',
  {'entities': [(14, 40, 'FOOD')]}),
 ('the chicken soup with noodles was fire', {'entities': [(4, 29, 'FOOD')]}),
 ('double ka meetha are amazing, you have to try it!',
  {'entities': [(0, 16, 'FOOD')]}),
 ('everytime i visit, i always get skillet cinnamon roll pint',
  {'entities': [(32, 58, 'FOOD')]}),
 ('my friends and i are never getting xuan asian bowl again',
  {'entities': [(35, 50, 'FOOD')]}),
 ('this place always messes up my order of red bull sugar free',
  {'entities': [(40, 59, 'FOOD')]}),
 ('i think that pork teriyaki is the best thing on the menu',
  {'entities': [(13, 26, 'FOOD')]}),
 ('i like to eat dr pepper bbq ribs family meal at this joint',
  {'entities': [(14, 44, 'FOOD')]}),
 ('the baked chocolate cookie topped with ice cream was fire',
  {'entities': [(4, 48, 'FOOD')]}),
 ('banana smoothie are amazing, you have to try it!',
  {'entities': [(0, 15, 'FOOD')]}),
 ('everytime i visit, i always get gallon u

In [15]:
# # GOD TIER COPY
# TRAIN_DATA = [
#     ('I like to eat hot pepper chicken nuggets at this joint', {
#         'entities': [(14, 40, 'FOOD')]
#     }),
#      ('The Turkey tacos was fire', {
#         'entities': [(4, 16, 'FOOD')]
#     }),
#     ('Pork Ribs You have to try it!', {
#         'entities': [(0, 9, 'FOOD')]
#     })
# ]

## Define variables

In [45]:
model = None
output_dir=Path("/Users/rahul/Desktop/entity/output_dir")
n_iter=10

## Load the model

In [46]:
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

Created blank 'en' model


## Setup pipeline

In [47]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

## Train the recognizer

In [49]:
for _, annotations in TRAIN_DATA:
    print(_, annotations)

    for ent in annotations.get('entities'):
        print(ent)
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            nlp.update(
                [text],  
                [annotations],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)

  4%|▍         | 2/50 [00:00<00:02, 17.24it/s]

my friends and i are never getting cheesy broccoli again {'entities': [(35, 50, 'FOOD')]}
(35, 50, 'FOOD')
my friends and i are never getting jamocha shake again {'entities': [(35, 48, 'FOOD')]}
(35, 48, 'FOOD')
hatch green chile cheeseburger are amazing, you have to try it! {'entities': [(0, 30, 'FOOD')]}
(0, 30, 'FOOD')
everytime i visit, i always get skillet cinnamon roll pint {'entities': [(32, 58, 'FOOD')]}
(32, 58, 'FOOD')
banana smoothie are amazing, you have to try it! {'entities': [(0, 15, 'FOOD')]}
(0, 15, 'FOOD')
everytime i visit, i always get shredded pork stomach with cilantro {'entities': [(32, 67, 'FOOD')]}
(32, 67, 'FOOD')
double ka meetha are amazing, you have to try it! {'entities': [(0, 16, 'FOOD')]}
(0, 16, 'FOOD')
buffalo tender wrap are amazing, you have to try it! {'entities': [(0, 19, 'FOOD')]}
(0, 19, 'FOOD')
i think that original chicken finger platter is the best thing on the menu {'entities': [(13, 44, 'FOOD')]}
(13, 44, 'FOOD')
i like to eat iced clover br

 10%|█         | 5/50 [00:00<00:00, 45.30it/s]

{'ner': 21.227732883635802}


 12%|█▏        | 6/50 [00:00<00:00, 54.38it/s]

...

{'ner': 4.676873670467758}


 10%|█         | 5/50 [00:00<00:01, 41.91it/s]

{'ner': 5.776936315771391}


100%|██████████| 50/50 [00:01<00:00, 40.37it/s]

{'ner': 6.140320056753647}


## Test the trained recognizer

In [80]:
import pickle
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import sklearn_crfsuite
import joblib

In [78]:
with open('final_model.pkl', 'rb') as fp:
    sent_final_2 = pickle.load(fp)

UnicodeDecodeError: 'ascii' codec can't decode byte 0xcc in position 5: ordinal not in range(128)

In [58]:
df = pd.read_csv('reviews.csv')

In [68]:
df.head()

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,0,xQY8N_XvtGbearJ5X4QryQ,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2,5,0,0,"As someone who has worked with many museums, I...",2015-04-15 05:21:16
1,1,UmFMZ8PyXZTY2QcwzsfQYA,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1,1,1,0,I am actually horrified this place is still in...,2013-12-07 03:16:52
2,2,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,I love Deagan's. I do. I really do. The atmosp...,2015-12-05 03:18:11
3,3,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1,0,0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",2011-05-27 05:30:52
4,4,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4,0,0,0,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57


In [65]:
TEST_DATA = list(df.loc[0:3,'text'])

In [66]:
TEST_DATA

['As someone who has worked with many museums, I was eager to visit this gallery on my most recent trip to Las Vegas. When I saw they would be showing infamous eggs of the House of Faberge from the Virginia Museum of Fine Arts (VMFA), I knew I had to go!\n\nTucked away near the gelateria and the garden, the Gallery is pretty much hidden from view. It\'s what real estate agents would call "cozy" or "charming" - basically any euphemism for small.\n\nThat being said, you can still see wonderful art at a gallery of any size, so why the two *s you ask? Let me tell you:\n\n* pricing for this, while relatively inexpensive for a Las Vegas attraction, is completely over the top. For the space and the amount of art you can fit in there, it is a bit much.\n* it\'s not kid friendly at all. Seriously, don\'t bring them.\n* the security is not trained properly for the show. When the curating and design teams collaborate for exhibitions, there is a definite flow. That means visitors should view the a

In [67]:
# TEST_DATA = [
#     ('Anytime I am in Chicago I always get the phones, but this time I chose to get the laptops and I loved it', {
#         'entities': [(0, 25, 'FOOD')]
#     }),
#      ('i like to eat butter chicken', {
#         'entities': [(0, 12, 'FOOD')]
#     }),
#     ('The double ka meetha is awesome here', {
#         'entities': [(0, 9, 'FOOD')]
#     })
# ]
for text in TEST_DATA:
    doc = nlp(text)
    #print(doc)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    #print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('someone who', 'FOOD'), ('worked with many museums', 'FOOD'), ('Vegas. When', 'FOOD'), ('would be showing infamous eggs', 'FOOD'), ('House of Faberge from', 'FOOD'), ('Virginia Museum', 'FOOD'), ('Fine Arts (VMFA)', 'FOOD'), ('real estate', 'FOOD'), ('small.', 'FOOD'), ('being said', 'FOOD'), ('still see wonderful', 'FOOD'), ('*s', 'FOOD'), ('relatively inexpensive', 'FOOD'), ('much.\n*', 'FOOD'), ('kid friendly', 'FOOD'), ('show.', 'FOOD'), ('design teams collaborate', 'FOOD'), ('definite flow. That means visitors should view', 'FOOD'), ('historical period', 'FOOD'), ('cultural significance (', 'FOOD'), ('helpful.', 'FOOD'), ('just look', 'FOOD'), ('whatever."', 'FOOD'), ('lack of knowledge', 'FOOD')]
Entities [('business.', 'FOOD'), ('son needed', 'FOOD'), ('past summer', 'FOOD'), ('$7 kids cut signs got', 'FOOD'), ('door. We', 'FOOD'), ('minutes as both stylists were', 'FOOD'), ('people.', 'FOOD'), ('. It', 'FOOD'), ('sofa they', 'FOOD'), ('with giant holes', 'FOOD'), ('s

In [ ]:
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    #print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

## Save model

In [ ]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

## Test the saved model

In [ ]:
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
for text, _ in TRAIN_DATA:
    doc = nlp2(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])